<h1>Wild Card</h1>

<h5>Bigram</h5>

In [1]:
def generateBigram(_queri):
    hasil = []
    for i in range(len(_queri)-1):
        if i == 0:
            hasil.append('$'+_queri[i])
        else:
            hasil.append(_queri[i] + _queri[i+1])
            
    return hasil

In [2]:
def wildCard(_queri):
    queri = _queri.replace('*','')
    bigram = generateBigram(queri)
    
    return bigram

<h1>Koreksi Ejaan</h1>

<h5>Kata Terisolasi</h5>

In [3]:
#from nltk.corpus import words #kamus inggris
from nltk.metrics import * #hitung jarak

In [4]:
def spellingCorection(_word):
    wordsSelected = identicalWord(_word)
    
    thresholdDistance = 3 #threshold
    mostIdenticalWord = []
    for i in wordsSelected: #check dengan levenshtaien
        tempDistance = edit_distance(i, _word)
        if tempDistance < thresholdDistance:
            sortestDistance = tempDistance
            mostIdenticalWord.append(i)
            
    return mostIdenticalWord

In [5]:
def identicalWord(_word):
    wordLenght = len(_word)
    wordsSelected = [] #word yang selisih panjang = 0
    
#     listOfWord = getListOfWord('hasil_inverted.txt')
    
    for word in listOfWordOfInvIndex: #tadinya listOfWord (yg di atas)
        if abs(len(word) - wordLenght) == 0 and word[0] == _word[0]: #panjangnya sama dan huruf depan sama
            wordsSelected.append(word)
            
    return wordsSelected

In [6]:
def getListOfWord(_path):
    f = open(_path,"r")
    words = f.read()
    words = words.replace('\t','')
    listOfWord = words.split('\n')
    
    return listOfWord

In [7]:
# a = spellingCorection('for me')
# print(a)

<h1>Pre-Procesing Queri</h1>

In [8]:
from nltk.stem import WordNetLemmatizer

In [9]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        if listOfWords[i] != 'OR' and listOfWords[i] != 'AND' and listOfWords[i] != 'NOT':
            listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

In [10]:
lemmatizer = WordNetLemmatizer()

def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

In [11]:
def tokenize(_queri):
    listOfWord = _queri.split(' ')
    
    return listOfWord

In [12]:
from nltk.stem.snowball import SnowballStemmer

def stemming(listOfWords):
    stemmer = SnowballStemmer("english")
#     stemmed = [stemmer.stem(word) for word in listOfWords]
    for i in range(len(listOfWords)):
        if listOfWords[i] != 'OR' and listOfWords[i] != 'AND' and listOfWords[i] != 'NOT':
            listOfWords[i] = stemmer.stem(listOfWords[i])
    
    return listOfWords

In [13]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

In [14]:
def preProcessingQuery(_query):
    listOfWords = tokenize(_query)
#     print('token : ',listOfWords)
    listOfWords = caseFolding(listOfWords)
#     print('case : ',listOfWords)
    listOfWords = removeStopWord(listOfWords)
#     print('remove : ',listOfWords)
    listOfWords = stemming(listOfWords)
#     print('stem : ',listOfWords)
    listOfWords = lemmatization(listOfWords)
#     print('lemma : ',listOfWords)
    
    return listOfWords

<h1>Main</h1>

In [15]:
def findBoolean(_listOfWord, _bool):
    boolIndex = []
#     listOfWord = _query.split(' ')
    for i in range(len(_listOfWord)):
        if _listOfWord[i] == _bool:
            boolIndex.append(i)
            
    return boolIndex

In [16]:
def isWordInTerm(_listOfWord):
    newListOfWord = []
    temp = ''
    for i in _listOfWord:
        if i != 'OR' and i != 'AND' and i != 'NOT':
            if i not in listOfWordOfInvIndex:
#                 print(i)
                try:
                    temp = spellingCorection(i)[0]
                    newListOfWord.append(temp)
                except:
                    print("No '",i,"' in Term")
#                     newListOfWord.append(temp)
            else:
                newListOfWord.append(i)
        else:
            newListOfWord.append(i)
                
    return newListOfWord

In [1]:
def queryProcessing(_query):
    andWord = []
    orWord = []
    notWord = []
    
    tempListOfWord = preProcessingQuery(_query) #pre processing query
#     print('pre : ', tempListOfWord)
    tempListOfWord = isWordInTerm(tempListOfWord) #koreksi query 
#     print('isword : ', tempListOfWord)
    listOfWord = []
    listOfWord = [tempListOfWord[0]]
    
    for i in range(1,len(tempListOfWord)): #mengubah NOT menjadi ~
        if tempListOfWord[i] == 'NOT':
            listOfWord.append('~' + tempListOfWord[i+1])
        elif tempListOfWord[i-1] != 'AND' and tempListOfWord[i] != 'AND' and tempListOfWord[i-1] != 'NOT':
            listOfWord.append('OR')
            listOfWord.append(tempListOfWord[i])
        elif tempListOfWord[i-1] != 'NOT':
            listOfWord.append(tempListOfWord[i])
        
            
    if '*' in _query:
        query = wildCard(_query)
    else:
#         if 'AND' in _query:
#             andIndex = findBoolean(listOfWord, 'AND')
            
#             for i in andIndex:
#                 andWord.append(listOfWord[i-1])
#                 andWord.append(listOfWord[i+1])
                    
#         if 'NOT' in _query:
#             for i in listOfWord:
#                 if i[0] == '~':
#                     notWord.append(i)
                    
        if 'AND' not in _query and 'OR' not in _query:
            orWord = listOfWord
#         elif 'OR' in _query:
#             orIndex = findBoolean(listOfWord, 'OR')
#             for i in orIndex:
#                 orWord.append(listOfWord[i-1])
#                 orWord.append(listOfWord[i+1])
#         else:
#             for i in range(0,len(listOfWord)-1):
#                 if listOfWord[i+1] != 'AND' and listOfWord[i] != 'AND':
#                     orWord.append(listOfWord[i])
#                     orWord.append(listOfWord[i+1])
                
#         print('OR : ',orWord)
#         print('AND : ',andWord)
#         print('NOT : ',notWord)
    return listOfWord 

In [29]:
listOfWordOfInvIndex = getListOfWord('hasil_inverted.txt')
queryProcessing('makan minum AND mandi')

['manag', 'OR', 'minim', 'AND', 'manag']


In [ ]:
makan AND minum
makan AND NOT minum
makan AND minum mandi
makan minum AND mandi
makan NOT minum AND mandi
NOT makan minum AND mandi (x)
makan OR NOT minum AND NOT main OR loker